In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Masking
from keras.layers import Attention, LayerNormalization, Dropout
from keras.optimizers import Adam
import os
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/My Drive/Colab Notebooks/Deep/English_Sinhala_Transfomer")
os.getcwd()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/Deep/English_Sinhala_Transfomer'

In [ ]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Deep/English_Sinhala_Transfomer/sinhala.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/Deep/English_Sinhala_Transfomer/sinhala.csv'

In [ ]:
spa_sentences = df["sinhala"] + "\t" + df["english"]

# Save the concatenated sentences to a text file
with open("spa.txt", "w", encoding="utf-8") as file:
    for sentence in spa_sentences:
        file.write(str(sentence) + "\n")

In [ ]:
text_file ="spa.txt"

In [ ]:
text_file ="spa.txt"
with open(text_file) as f:
  lines=f.read().split("\n")[:-1]

In [ ]:
i=0
for line in lines:
  print(line)
  i=i+1
  if(i==20):
    break

ඔයාට තැපෑලෙන් පැකේජ් එකක් හම්බවේවි	you will receive a package in the mail
ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කිරීමේ කේතයක්  හම්බවේවි	you will receive a confirmation code after completing the registration
ඊලඟ මිලදී ගැනීම කරන කොට ඔයාට වට්ටමක් හම්බවේවි	you will receive a discount on your next purchase
ඔයාට අපේ පාරිභෝගික සේවා කණ්ඩායමෙන් දුරකථන ඇමතුමක් හම්බවේවි	you will receive a phone call from our customer service team
ඔයාගේ ඇණවුම සූදානම් උනාට පස්සෙ ඔයාට දැනුම් දීමක් හම්බවේවි	you will receive a notification when your order is ready for pickup
ඔයාට පැය 24ක් ඇතුලත ඔයාගෙ විමසීමට පිළිතුරක්  හම්බවේවි	you will receive a response to your inquiry within 24 hours
ඔයාට ඔයාගේ පක්ෂපාතිත්වය වෙනුවෙන් තෑග්ගක් හම්බවේවි	you will receive a gift for your loyalty
ඔයාට උත්සවයට ආරාධනාවක් හම්බවේවි	you will receive an invitation to the event
ඔයාට රිටන් කරන භාණ්ඩය වෙනුවෙන් ආපසු මුදල් ගෙවීමක් හම්බවේවි	you will receive a refund for the returned item
ඔයාට විස්තර එක්ක  ඊමේල් එකක් හම්බවේවි	you'll receive an e

In [ ]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

එක් කලෙක කුඩා ගමක ලිලී නම් තරුණියක් ජීවත් වූ අතර ඇයට ලස්සනම මල් වගා කිරීම සඳහා විශේෂ තෑග්ගක් තිබුණි	Once upon a time in a small village there lived a young girl named Lily who had a special gift for growing the most beautiful flowers
දුර රටක ශ්‍රීමත් ආතර් නම් නිර්භීත නයිට්වරයා නපුරු මායාකාරයෙකුගෙන් පැහැරගෙන ගිය කුමරියක් බේරා ගැනීමට භයානක ගමනක් ආරම්භ කළේය	In a distant land a brave knight named Sir Arthur embarked on a perilous journey to rescue a kidnapped princess from an evil sorcerer
කුතුහලය දනවන ගවේෂකයෙකු වන එමිලි ඇමසන් වනාන්තරයේ ගැඹුරින් ගිලිහී ගිය පැරණි නගරයක සැඟවුණු වස්තු සොයා ගැනීමට ත්‍රාසජනක ගමනක් ආරම්භ කළාය	Emily a curious explorer set out on an adventure to uncover the hidden treasures of an ancient lost city deep in the Amazon rainforest
සාමකාමී නගරයක් වන විලෝබෲක්හි ඔලිවර් නම් දඟකාර බළලාට විනෝදජනක සහ අනපේක්ෂිත දුෂ්කරතාවන්ට පත්වීමේ දක්ෂතාවයක් තිබුණි	In the peaceful town of Willowbrook a mischievous cat named Oliver had a talent for getting into amusing and unexpected predicam

In [ ]:
import random
import string
import re

text_pairs = []


for line in lines:
    if '\t' in line:
        english, sinhala = line.split("\t")
        sinhala = "[start]" + sinhala.strip() + "[end]"
        text_pairs.append((english.strip(), sinhala))
for i in range(3):
    print(random.choice(text_pairs))


('සමහරවිට එයාට මතක නැති\u200d හේතුව එයා කරපු දේ ඇත්ත නිසා වෙන්න ඇති', "[start]maybe the reason he doesn't remember is because what he did was true[end]")
('නෑ මට සමාවෙන්න', "[start]not i'm very sorry[end]")
('වැඩි වෙලාවක් ඉගෙන ගන්න යෙදෙව්වානම්', '[start]if you spent more time studying[end]')


In [ ]:
import random
random.shuffle(text_pairs)

In [ ]:
num_val_samples =int(0.15*len(text_pairs))
num_train_samples=len(text_pairs) -2* num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [ ]:
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))

Total sentences: 80633
Training set size: 56445
Validation set size: 12094
Testing set size: 12094


In [ ]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

80633

In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [ ]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
    lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length + 1,
  standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]

In [ ]:
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [ ]:
batch_size=64

def format_dataset(eng,spa):
  eng=source_vectorization(eng)
  spa=target_vectorization(spa)
  return({
      "english":eng,
      "sinhala":spa[:,:-1],
  },spa[:,1:])

def make_dataset(pairs):
  eng_texts, spa_texts =zip(*pairs)
  eng_texts =list(eng_texts)
  spa_texts =list(spa_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(format_dataset, num_parallel_calls=4)
  return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

for inputs, targets in train_ds.take(1):
  print(f"inputs['english'].shape:{inputs['english'].shape}")
  print(f"inputs['sinhala'].shape:{inputs['sinhala'].shape}")
  print(f"targets.shape:{targets.shape}")

  inputs['english'].shape:  (64, 20)
  inputs['sinhala'].shape: (64, 20)
  targets.shape: (64, 20)
print(list(train_ds.as_numpy_iterator())[50])

inputs['english'].shape:(64, 20)
inputs['sinhala'].shape:(64, 20)
targets.shape:(64, 20)
({'english': array([[   3,  270,  932, ...,    0,    0,    0],
       [ 182,   48,  219, ...,    0,    0,    0],
       [ 691, 1775,    0, ...,    0,    0,    0],
       ...,
       [  59, 2934,    0, ...,    0,    0,    0],
       [   3,    7,   78, ...,    0,    0,    0],
       [   9,   34,   23, ...,    0,    0,    0]]), 'sinhala': array([[ 300,    2,    9, ...,    0,    0,    0],
       [  98,  261,   23, ...,    0,    0,    0],
       [7023,    1,    0, ...,    0,    0,    0],
       ...,
       [ 584,   13, 1689, ...,    0,    0,    0],
       [  14,   55,   13, ...,    0,    0,    0],
       [  58,   27,  121, ...,    0,    0,    0]])}, array([[   2,    9,   31, ...,    0,    0,    0],
       [ 261,   23,  404, ...,    0,    0,    0],
       [   1,    0,    0, ...,    0,    0,    0],
       ...,
       [  13, 1689,    0, ...,    0,    0,    0],
       [  55,   13, 2417, ...,    0,    0,    

In [ ]:

import string
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask
        )
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


In [ ]:
embed_dim = 256
dense_dim = 2048
num_heads = 8


# Define the encoder inputs
encoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="english")

# Add positional embedding to the encoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

# Encode the inputs using TransformerEncoder
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# Define the decoder inputs
decoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="sinhala")

# Add positional embedding to the decoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

# Decode the inputs using TransformerDecoder
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)

# Apply dropout
x = layers.Dropout(0.5)(x)

# Generate decoder outputs
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

# Create the transformer model
transformer = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

In [ ]:
transformer.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])

transformer.fit(train_ds, epochs=30, validation_data=val_ds)


Epoch 1/30
882/882 [==============================] - 78s 78ms/step - loss: 5.0536 - accuracy: 0.2915 - val_loss: 4.1291 - val_accuracy: 0.3758
Epoch 2/30
882/882 [==============================] - 60s 69ms/step - loss: 4.1761 - accuracy: 0.3846 - val_loss: 3.7370 - val_accuracy: 0.4247
Epoch 3/30
882/882 [==============================] - 60s 69ms/step - loss: 3.8039 - accuracy: 0.4323 - val_loss: 3.5670 - val_accuracy: 0.4527
Epoch 4/30
882/882 [==============================] - 65s 73ms/step - loss: 3.5390 - accuracy: 0.4684 - val_loss: 3.4973 - val_accuracy: 0.4647
Epoch 5/30
882/882 [==============================] - 61s 69ms/step - loss: 3.3383 - accuracy: 0.4955 - val_loss: 3.4583 - val_accuracy: 0.4736
Epoch 6/30
882/882 [==============================] - 61s 69ms/step - loss: 3.1808 - accuracy: 0.5193 - val_loss: 3.4867 - val_accuracy: 0.4769
Epoch 7/30
882/882 [==============================] - 61s 69ms/step - loss: 3.0576 - accuracy: 0.5391 - val_loss: 3.4654 - val_accuracy:

In [ ]:
import numpy as np

# Define the vocabulary and index lookup for Spanish
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))


-
එතකොට ඩැඩී ගෙදර එනවාද
[start] on [UNK] is coming[end]                
-
ඇය මැඩපවත්වගෙන ඉන්නේ වෙන කෙනෙක් එක්ක සම්බන්ධ නිසා
[start] from im with another person[end]               
-
මට ඕනේ අපේ NSG කමාන්ඩෝස්ලා ජනාධිපතිතුමාවම ෆෝකස් කරන් ඉන්නවට
[start] from [UNK] our [UNK] will [UNK] will be taken away[end]          
-
ඒ කිව්ව්
[start] from that[end]                  
-
මේ පාර ඔයාලා දෙන්නාම
[start] two of you both of you[end]              
-
මම පොරොන්දු වෙනවා මේකෙන් පස්සේ
[start] i promise[end]                  
-
සජීවීව අපේ චැනල් එකේ
[start] on our channel[end]                 
-
මගේ පුතා
[start] son[end]                   
-
කොහොමද ඔයාට
[start] how you[end]                  
-
ඉතිං ඒ ඔයාලා නේද     නෑ
[start] thats isnt it[end]                 
-
ඝර්ෂණ නිරෝධය ඒක තමයි විදිය
[start] way it was the way to [UNK]             
-
හායි ඇලිසන්
[start] [UNK]                   
-
කම්මුල දෙන්න
[start] from [UNK]                  
-
මම දෙන්නම් තමුසෙට දර්ශනයක්
[start] will give me [UNK]               

In [ ]:
import numpy as np


sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print("English: ", input_sentence)
    print("Sinhala: ", decode_sequence(input_sentence))


-
English:  එහෙනම් කන්න එපා
Sinhala:  [start] eat no[end]                  
-
English:  බීබොට්
Sinhala:  [start]                    
-
English:  හොඳයි මෑම්
Sinhala:  [start] maam[end]                   
-
English:  පරිවර්තනය සහ උපසිරැසි
Sinhala:  [start] from [UNK]                  
-
English:  කවුද අහන්නේ
Sinhala:  [start] ask[end]                   
-
English:  අන්ශ්යමු
Sinhala:  [start] from [UNK]                  
-
English:  ඒකෙන් හොයන්න තියෙන ඉඩ අඩු කරනවා නේද
Sinhala:  [start] from matter what you [UNK]               
-
English:  මට අවකලණ සමීකරණ පන්තිය තියෙනවා තව විනාඩි 15කින්
Sinhala:  [start] from [UNK] minutes in class will meet another same now[end]          
-
English:  ඔයාට ඔයාගේ පක්ෂපාතිත්වය වෙනුවෙන් තෑග්ගක් හම්බවේවි
Sinhala:  [start] from your gift for your gift for it[end]            
-
English:  අවසන් දිනය
Sinhala:  [start] at [UNK]                  
-
English:  මොනාද අනේ
Sinhala:  [start] please[end]                   
-
English:  හෙස්පෙරා මේ ගහ දිහා බලන්න
Sinhala:  [s

In [ ]:
import numpy as np


sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

# Get English input from the user
input_sentence = input("Enter an English sentence: ")

# Translate the input sentence to Sinhala
translated_sentence = decode_sequence(input_sentence)

# Print the translated sentence
print("Sinhala translation:", translated_sentence)


Enter an English sentence: කොහොමද ඔයාට
Sinhala translation: [start] how you[end]                  
